In [ ]:
from random import choice, random
import string

target = list("print('hello world')")
alphabet = string.ascii_lowercase + " " + "'" + "(" + ")"

pmut = 0.05
nchildren = 100

def fitness(trial):
    return sum(t != h for t,h in zip(trial, target))

def mutate(parent):
    return [(choice(alphabet) if random() < pmut else ch) for ch in parent]

parent = [choice(alphabet) for _ in range(len(target))]
igen = 0
while parent != target:
    children = (mutate(parent) for _ in range(nchildren))
    parent = min(children, key=fitness)
    print(f"{igen} gen: {''.join(parent)}")
    igen += 1

### Geneticke programovani

In [ ]:
import operator
import math
import random

import numpy

!pip install deap
from deap import algorithms #obsahuje algoritmz EA, GA atd.
from deap import base       #obsahuje nastroje pro prirazeni funkci do operatoru, kompilaci, inicializace populace
from deap import creator    #propojuje stromovou strukturu s fitness funkci
from deap import tools      #obsahuje nastroje pro mereni statistik
from deap import gp         #nastroje pro vytvoreni jedince jako stromu do GP



#chranene deleni proti cislo/0
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

#vytvoreni stromu, koren stromu (MAIN), ma aritu = 1 (jeden vstup do funkce)
pset = gp.PrimitiveSet("MAIN", 1)

#pridani primitiv pro soucet, rozdil, soucin, chranene deleni (binarni operace)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)

#pridani primitiv pro negaci (zaporne cislo), cos a sin (unarni operace)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)

#nahodne cislo -1, 0, 1, ktere se meni, kdykoliv ho pridame do stromu
pset.addEphemeralConstant("rand101", lambda: random.randint(-1,1))

#defaultni nazev vstupu funkce prejmenujeme na x
pset.renameArguments(ARG0='x')



#nastaveni fitness fce: vahy zaporne, protoze minimalizujeme odchylku phi(x) - f(x)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

#propojeni stromu s fitness funkci, jedinci jsou obecna class Individual
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)



#nastaveni procesu genetickeho programovani
toolbox = base.Toolbox()

#jedinec bude tvoren ze stromu (genHalfAndHalf) o velikosti 1 nebo dva uzly
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)

#tento strom expr je muj jedinec
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

#populace je vytvarena z jedincu
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

#dej to cele dohromady (prevedeni do spustitelne formy)
toolbox.register("compile", gp.compile, pset=pset)



#optimalizujeme prolozeni 20 bodu kvarticke funkce f(x) = x**4 + x**3 + x**2 + x 
def evalSymbReg(individual, points):
    #premena stromu na volatelnou aproximacni funkci
    func = toolbox.compile(expr=individual)
    #chyba (phi(x)-f(x))**2
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    #vratime chybu souctu ctvercu, pozor evalute fce musi vracet n-tici, jinak nedefinovane problemy
    return math.fsum(sqerrors) / len(points),

#- jak se pocita fitness funkce
toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])

#- zaregistruj pod terminem select operator selekce a nastav velikost turnaje 3 
#(https://www.geeksforgeeks.org/tournament-selection-ga/)
toolbox.register("select", tools.selTournament, tournsize=3)

#- nastaveni reprodukcniho operatoru, jednobodove krizeni s uniformacni sanci, ze uzel stromu bude co-point
toolbox.register("mate", gp.cxOnePoint)

#nastaveni podstromu pro mutaci, jedna se o cely podstrom
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)

#nastaveni mutacniho operatoru, mututuje se tak, cely podstrom expr_mut se appenduje do uzlu
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

#zavedeni hloubkovych limitu operatoru, Koza doporucuje 17
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))



def main():

    #pocatecni populace
    pop = toolbox.population(n=300)

    #struktura obsahujici nejlepsi kandidaty
    hof = tools.HallOfFame(1)
    
    #jake vsechny statistiky chceme pocitat (chceme pocitat fitness a velikost jedince)
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)

    #na techto statistickach chceme spocitat prumer, s.odch., min a max
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    print(hof.items[0])

if __name__ == "__main__":
    main()